## SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 提取特征
train_features, train_labels = extract_features(train_loader, model, device)
validation_features, validation_labels = extract_features(validation_loader, model, device)
test_features, test_labels = extract_features(test_loader, model, device)

# 训练 SVM
svm_clf = SVC(kernel='linear', C=1, probability=True)  # 设置 probability=True 以输出概率
svm_clf.fit(train_features, train_labels)

# 验证 SVM
validation_predictions_svm = svm_clf.predict(validation_features)
validation_accuracy_svm = accuracy_score(validation_labels, validation_predictions_svm)
print(f'SVM 验证准确率: {validation_accuracy_svm:.4f}')

# 测试 SVM
test_predictions_svm = svm_clf.predict(test_features)
test_accuracy_svm = accuracy_score(test_labels, test_predictions_svm)
print(f'SVM 测试准确率: {test_accuracy_svm:.4f}')

## MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

# 训练 MLP
mlp_clf = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
mlp_clf.fit(train_features, train_labels)

# 验证 MLP
validation_predictions_mlp = mlp_clf.predict(validation_features)
validation_accuracy_mlp = accuracy_score(validation_labels, validation_predictions_mlp)
print(f'MLP 验证准确率: {validation_accuracy_mlp:.4f}')

# 测试 MLP
test_predictions_mlp = mlp_clf.predict(test_features)
test_accuracy_mlp = accuracy_score(test_labels, test_predictions_mlp)
print(f'MLP 测试准确率: {test_accuracy_mlp:.4f}')

## XGB

In [ ]:
import xgboost as xgb

# 转换数据为 DMatrix 格式
dtrain = xgb.DMatrix(train_features, label=train_labels)
dvalid = xgb.DMatrix(validation_features, label=validation_labels)
dtest = xgb.DMatrix(test_features, label=test_labels)

# 设置参数
params = {
    'objective': 'multi:softprob',  # 用于多分类
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'max_depth': 6,
    'eta': 0.1,
    'seed': 42
}

# 训练 XGBoost
xgb_clf = xgb.train(params, dtrain, num_boost_round=100, evals=[(dvalid, 'validation')])

# 验证 XGBoost
validation_predictions_xgb = xgb_clf.predict(dvalid)
validation_accuracy_xgb = accuracy_score(validation_labels, np.argmax(validation_predictions_xgb, axis=1))
print(f'XGBoost 验证准确率: {validation_accuracy_xgb:.4f}')

# 测试 XGBoost
test_predictions_xgb = xgb_clf.predict(dtest)
test_accuracy_xgb = accuracy_score(test_labels, np.argmax(test_predictions_xgb, axis=1))
print(f'XGBoost 测试准确率: {test_accuracy_xgb:.4f}')

## Soft Voting

In [ ]:
from sklearn.ensemble import VotingClassifier

# 创建 VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('svm', svm_clf),
        ('mlp', mlp_clf),
        ('xgb', xgb.XGBClassifier(objective='multi:softprob', n_estimators=100, max_depth=6, learning_rate=0.1))
    ],
    voting='soft',  # 使用软投票
    weights=[1, 1, 1]  # 根据需要调整权重
)

# 训练 VotingClassifier
voting_clf.fit(train_features, train_labels)

# 在测试集上进行预测
test_predictions_voting = voting_clf.predict(test_features)

# 计算最终准确率
voting_accuracy = accuracy_score(test_labels, test_predictions_voting)
print(f'Voting Classifier 测试准确率: {voting_accuracy:.4f}')